In [ ]:
from google.colab import auth, drive

drive.mount('/content/drive')
print('Mounted')

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pickle

In [ ]:
event_df = pd.read_csv('/content/drive/MyDrive/의료인공지능/mimic_test/df_event.csv')
non_event_df = pd.read_csv('/content/drive/MyDrive/의료인공지능/mimic_test/df_non_event.csv')

event_chartdf = pd.read_csv('/content/drive/MyDrive/의료인공지능/mimic_test/chartdf_event.csv')
non_event_chartdf = pd.read_csv('/content/drive/MyDrive/의료인공지능/mimic_test/chartdf_non_event.csv')

##Non event data 만들기

In [ ]:
 with open('/content/drive/MyDrive/의료인공지능/mimic_test/crystalloid_df_non_event.pickle', 'rb') as f:
     crystalloid_df = pickle.load(f)

 with open('/content/drive/MyDrive/의료인공지능/mimic_test/colloid_df_non_event.pickle', 'rb') as f:
     colloid_df = pickle.load(f)

 with open('/content/drive/MyDrive/의료인공지능/mimic_test/rbc_df_non_event.pickle', 'rb') as f:
     rbc_df = pickle.load(f)

 with open('/content/drive/MyDrive/의료인공지능/mimic_test/urine_df_non_event.pickle', 'rb') as f:
     urine_df = pickle.load(f)

In [ ]:
df = non_event_df
chart_df = non_event_chartdf

In [ ]:
## drop outlier

sbp = [220050, 220179, 225309, 51, 455]
dbp = [220051, 220180, 225310, 8368, 8441]
resp = [220210, 618]
hr = [220045, 211] 
temp_c = [223762, 676, 677] 
spo2 = [646]

chart_df['valuenum'] = np.select(
    [
      chart_df['itemid'].isin(sbp) & (chart_df.valuenum > 0) & (chart_df.valuenum <= 300), 
      chart_df['itemid'].isin(dbp) & (chart_df.valuenum > 0) & (chart_df.valuenum <= 200),
      chart_df['itemid'].isin(resp) & (chart_df.valuenum > 11) & (chart_df.valuenum <= 80),
      chart_df['itemid'].isin(hr) & (chart_df.valuenum > 7) & (chart_df.valuenum <= 250),
      chart_df['itemid'].isin(temp_c) & (chart_df.valuenum > 12) & (chart_df.valuenum <= 150),
      chart_df['itemid'].isin(spo2) & (chart_df.valuenum > 13) & (chart_df.valuenum <= 90)
    ], 
    [
      chart_df['valuenum'],
      chart_df['valuenum'],
      chart_df['valuenum'],
      chart_df['valuenum'],
      chart_df['valuenum'],
      chart_df['valuenum']
    ], 
    default=np.nan
)

In [ ]:
def get_data_per_id(df,id):

  per_id_hour_lab= pd.DataFrame()
  df_trial= df[df['hadm_id']==id][["charttime","itemid",'valuenum']].drop_duplicates(keep = 'last') 
  chart_trial = df_trial.pivot(index='charttime', columns='itemid', values='valuenum')
  for column_name in chart_trial.columns:
    per_id_hour_lab= per_id_hour_lab.append(pd.Series(chart_trial.loc[:,column_name],name=column_name).resample('h').mean()) ## 이상치 제거 필요
  
  data_per_id = per_id_hour_lab.T
  data_per_id['hadm_id'] = id ## hadm_id 추가

  return data_per_id

In [ ]:
def check_number_col(df):

  sbp = [220050, 220179, 225309, 51, 455]
  dbp = [220051, 220180, 225310, 8368, 8441]
  resp = [220210, 618]
  hr = [220045, 211] 
  temp_c = [223762, 676, 677] 
  spo2 = [646] 

  num_col = 0
  lab_params = [sbp,dbp,resp,hr, temp_c, spo2]
  for params in lab_params:
    if (any(i in params for i in list(df.columns))):
      num_col+=1
  return num_col

In [ ]:
def get_number_checked_data_per_id(total_df):

  id_list = []
  for id in sorted(list(set(total_df['hadm_id']))):
    if check_number_col(get_data_per_id(total_df, id)) >= 6:
      id_list.append(id)
  
  return id_list

In [ ]:
def combine_same_item(output):

  sbp = [220050, 220179, 225309, 51, 455]
  dbp = [220051, 220180, 225310, 8368, 8441]
  resp = [220210, 618]
  hr = [220045, 211] 
  temp_c = [223762, 676, 677] 
  spo2 = [646] 

  col_names = ['sbp','dbp','resp','hr', 'temp_c', 'spo2']
  lab_params = [sbp,dbp,resp,hr, temp_c, spo2]
  lab_dict = dict(zip(col_names, lab_params))

  for col, item_id in lab_dict.items():
    for i in range(len(item_id)):
      if item_id[i] in output.columns:
        output[col] = np.nan
        output[col] = np.where(output[col].isnull(), output[item_id[i]], output[col])

  var_list = ['hadm_id', 455, 8441, 618, 211, 677, 646]
  output = output[var_list]
  output = output.rename(columns = {455:'sbp',8441:'dbp',618:'resp',211:'hr',677:'temp_c',646:'spo2'})
                
  return output

In [ ]:
chart_df['charttime'] = pd.to_datetime(chart_df['charttime'])
chart_df['charttime_ind'] = pd.to_datetime(chart_df['charttime_ind'])

In [ ]:
id_list = get_number_checked_data_per_id(chart_df)

In [ ]:
output = pd.DataFrame()

for id in id_list:
  tmp_output = get_data_per_id(chart_df,id)
  output = tmp_output.append(output)

In [ ]:
result = combine_same_item(output)
var_list = ['hadm_id', 'sbp', 'dbp', 'resp', 'hr', 'temp_c', 'spo2']
result = result.loc[:,var_list]

In [ ]:
df['charttime'] = pd.to_datetime(df['charttime'])

In [ ]:
df_schema = pd.DataFrame()

for i in range(len(df)):
  tmp_output = pd.DataFrame({'hadm_id': df['hadm_id'][i], 
                     'time': pd.date_range(df['charttime'][i]-timedelta(1), df['charttime'][i] , freq="60min")}
                    )
  tmp_output = tmp_output.iloc[1:]
  df_schema = tmp_output.append(df_schema)

In [ ]:
df_schema['time'] = df_schema['time'].dt.floor('H')
df_schema = df_schema.drop_duplicates(['hadm_id', 'time'], keep='first')
df_schema = df_schema.set_index(['hadm_id', 'time'])

In [ ]:
result.rename_axis('time', inplace=True)
result.reset_index(inplace=True)
result = result.set_index(['hadm_id', 'time'])

In [ ]:
result_combined = df_schema.merge(result, left_index=False, right_index=True, left_on= ['hadm_id', 'time'] ,how = 'left')
result_combined['event'] = 0

In [ ]:
def matching_schema(df):

  df = df.dropna()
  df = df.drop_duplicates()
  df = df.assign(charttime_ind=df.charttime.dt.round('H'))

  df['charttime_ind'] = df['charttime'].dt.round('H')
  df.rename(columns={'charttime_ind': 'time'}, inplace = True)
  df = df.drop('charttime',axis=1)
  df = df.set_index(['hadm_id','time'])

  return df

In [ ]:
result_combined_r = result_combined.reset_index(drop=False, col_fill='index')

In [ ]:
df_list = [crystalloid_df, colloid_df, rbc_df, urine_df]

for i in range(len(df_list)):
  df = df_list[i]
  df = matching_schema(df)
  result_combined_r = result_combined_r.merge( df,  how='left', left_on=['hadm_id','time'], right_on = ['hadm_id','time'])

In [ ]:
result_combined_r = result_combined_r.drop_duplicates(['hadm_id','time'])

In [ ]:
## binary
result_combined_r['crystalloid_bolus_i'] = np.where(result_combined_r['crystalloid_bolus'].notnull(), 1,0)
result_combined_r['colloid_bolus_i'] = np.where(result_combined_r['colloid_bolus'].notnull(), 1,0)
result_combined_r['urine_i'] = np.where(result_combined_r['urine'].notnull(), 1,0)

result_combined_r['crystalloid_bolus'] = np.where(result_combined_r['crystalloid_bolus'].notnull(), result_combined_r['crystalloid_bolus'],0)
result_combined_r['colloid_bolus'] = np.where(result_combined_r['colloid_bolus'].notnull(), result_combined_r['colloid_bolus'],0)
result_combined_r['rbc'] = np.where(result_combined_r['rbc'].notnull(), result_combined_r['rbc'],0)
result_combined_r['urine'] = np.where(result_combined_r['urine'].notnull(), result_combined_r['urine'],0)

In [ ]:
result_combined_r = result_combined_r.set_index(['hadm_id','time'])

In [ ]:
# ## KNN
# from sklearn.impute import KNNImputer

# imputer = KNNImputer(n_neighbors=2)
# after_imputation = imputer.fit_transform(result_combined_r)

# result_combined_KNN = pd.DataFrame(after_imputation)
# result_combined_KNN.columns = result_combined_r.columns
# result_combined_KNN.index = result_combined_r.index
# result_combined_KNN.to_csv("/content/drive/MyDrive/의료인공지능/mimic_test/time_series_non_event_KNN.csv")

In [ ]:
# ## iterative
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# imputer = IterativeImputer()
# after_imputation = imputer.fit_transform(result_combined_r)

# result_combined_iter = pd.DataFrame(after_imputation)
# result_combined_iter.columns = result_combined_r.columns
# result_combined_iter.index = result_combined_r.index
# result_combined_iter.to_csv("/content/drive/MyDrive/의료인공지능/mimic_test/time_series_non_event_iter.csv")

In [ ]:
result_combined_r.to_csv("/content/drive/MyDrive/의료인공지능/mimic_test/time_series_non_event.csv")

## Event data 만들기

In [ ]:
 with open('/content/drive/MyDrive/의료인공지능/mimic_test/crystalloid_df_event.pickle', 'rb') as f:
     crystalloid_df = pickle.load(f)

 with open('/content/drive/MyDrive/의료인공지능/mimic_test/colloid_df_event.pickle', 'rb') as f:
     colloid_df = pickle.load(f)

 with open('/content/drive/MyDrive/의료인공지능/mimic_test/rbc_df_event.pickle', 'rb') as f:
     rbc_df = pickle.load(f)

 with open('/content/drive/MyDrive/의료인공지능/mimic_test/urine_df_event.pickle', 'rb') as f:
     urine_df = pickle.load(f)

In [ ]:
df = event_df
chart_df = event_chartdf

In [ ]:
## drop outlier

sbp = [220050, 220179, 225309, 51, 455]
dbp = [220051, 220180, 225310, 8368, 8441]
resp = [220210, 618]
hr = [220045, 211] 
temp_c = [223762, 676, 677] 
spo2 = [646]

chart_df['valuenum'] = np.select(
    [
      chart_df['itemid'].isin(sbp) & (chart_df.valuenum > 0) & (chart_df.valuenum <= 300), 
      chart_df['itemid'].isin(dbp) & (chart_df.valuenum > 0) & (chart_df.valuenum <= 200),
      chart_df['itemid'].isin(resp) & (chart_df.valuenum > 11) & (chart_df.valuenum <= 80),
      chart_df['itemid'].isin(hr) & (chart_df.valuenum > 7) & (chart_df.valuenum <= 250),
      chart_df['itemid'].isin(temp_c) & (chart_df.valuenum > 12) & (chart_df.valuenum <= 150),
      chart_df['itemid'].isin(spo2) & (chart_df.valuenum > 13) & (chart_df.valuenum <= 90)
    ], 
    [
      chart_df['valuenum'],
      chart_df['valuenum'],
      chart_df['valuenum'],
      chart_df['valuenum'],
      chart_df['valuenum'],
      chart_df['valuenum']
    ], 
    default=np.nan
)

In [ ]:
def get_data_per_id(df,id):

  per_id_hour_lab= pd.DataFrame()
  df_trial= df[df['hadm_id']==id][["charttime","itemid",'valuenum']].drop_duplicates(keep = 'last') 
  chart_trial = df_trial.pivot(index='charttime', columns='itemid', values='valuenum')
  for column_name in chart_trial.columns:
    per_id_hour_lab= per_id_hour_lab.append(pd.Series(chart_trial.loc[:,column_name],name=column_name).resample('h').mean()) ## 이상치 제거 필요
  
  data_per_id = per_id_hour_lab.T
  data_per_id['hadm_id'] = id ## hadm_id 추가

  return data_per_id

In [ ]:
def check_number_col(df):

  sbp = [220050, 220179, 225309, 51, 455]
  dbp = [220051, 220180, 225310, 8368, 8441]
  resp = [220210, 618]
  hr = [220045, 211] 
  temp_c = [223762, 676, 677] 
  spo2 = [646] 

  num_col = 0
  lab_params = [sbp,dbp,resp,hr, temp_c, spo2]
  for params in lab_params:
    if (any(i in params for i in list(df.columns))):
      num_col+=1
  return num_col

In [ ]:
def get_number_checked_data_per_id(total_df):

  id_list = []
  for id in sorted(list(set(total_df['hadm_id']))):
    if check_number_col(get_data_per_id(total_df, id)) >= 6:
      id_list.append(id)
  
  return id_list

In [ ]:
def combine_same_item(output):

  sbp = [220050, 220179, 225309, 51, 455]
  dbp = [220051, 220180, 225310, 8368, 8441]
  resp = [220210, 618]
  hr = [220045, 211] 
  temp_c = [223762, 676, 677] 
  spo2 = [646] 

  col_names = ['sbp','dbp','resp','hr', 'temp_c', 'spo2']
  lab_params = [sbp,dbp,resp,hr, temp_c, spo2]
  lab_dict = dict(zip(col_names, lab_params))

  for col, item_id in lab_dict.items():
    for i in range(len(item_id)):
      if item_id[i] in output.columns:
        output[col] = np.nan
        output[col] = np.where(output[col].isnull(), output[item_id[i]], output[col])

  var_list = ['hadm_id', 455, 8441, 618, 211, 677, 646]
  output = output[var_list]
  output = output.rename(columns = {455:'sbp',8441:'dbp',618:'resp',211:'hr',677:'temp_c',646:'spo2'})
                
  return output

In [ ]:
chart_df['charttime'] = pd.to_datetime(chart_df['charttime'])
chart_df['charttime_ind'] = pd.to_datetime(chart_df['charttime_ind'])

In [ ]:
id_list = get_number_checked_data_per_id(chart_df)

In [ ]:
output = pd.DataFrame()

for id in id_list:
  tmp_output = get_data_per_id(chart_df,id)
  output = tmp_output.append(output)

In [ ]:
result = combine_same_item(output)
var_list = ['hadm_id', 'sbp', 'dbp', 'resp', 'hr', 'temp_c', 'spo2']
result = result.loc[:,var_list]

In [ ]:
df['charttime'] = pd.to_datetime(df['charttime'])

In [ ]:
df_schema = pd.DataFrame()

for i in range(len(df)):
  tmp_output = pd.DataFrame({'hadm_id': df['hadm_id'][i], 
                     'time': pd.date_range(df['charttime'][i]-timedelta(1), df['charttime'][i] , freq="60min")}
                    )
  tmp_output = tmp_output.iloc[1:]
  df_schema = tmp_output.append(df_schema)

In [ ]:
df_schema['time'] = df_schema['time'].dt.floor('H')
df_schema = df_schema.drop_duplicates(['hadm_id', 'time'], keep='first')
df_schema = df_schema.set_index(['hadm_id', 'time'])

In [ ]:
result.rename_axis('time', inplace=True)
result.reset_index(inplace=True)
result = result.set_index(['hadm_id', 'time'])

In [ ]:
result_combined = df_schema.merge(result, left_index=False, right_index=True, left_on= ['hadm_id', 'time'] ,how = 'left')
result_combined['event'] = 1

In [ ]:
def matching_schema(df):

  df = df.dropna()
  df = df.drop_duplicates()
  df = df.assign(charttime_ind=df.charttime.dt.round('H'))

  df['charttime_ind'] = df['charttime'].dt.round('H')
  df.rename(columns={'charttime_ind': 'time'}, inplace = True)
  df = df.drop('charttime',axis=1)
  df = df.set_index(['hadm_id','time'])

  return df

In [ ]:
result_combined_r = result_combined.reset_index(drop=False, col_fill='index')

In [ ]:
df_list = [crystalloid_df, colloid_df, rbc_df, urine_df]

for i in range(len(df_list)):
  df = df_list[i]
  df = matching_schema(df)
  result_combined_r = result_combined_r.merge( df,  how='left', left_on=['hadm_id','time'], right_on = ['hadm_id','time'])

In [ ]:
result_combined_r = result_combined_r.drop_duplicates(['hadm_id','time'])

In [ ]:
## binary
result_combined_r['crystalloid_bolus_i'] = np.where(result_combined_r['crystalloid_bolus'].notnull(), 1,0)
result_combined_r['colloid_bolus_i'] = np.where(result_combined_r['colloid_bolus'].notnull(), 1,0)
result_combined_r['urine_i'] = np.where(result_combined_r['urine'].notnull(), 1,0)

result_combined_r['crystalloid_bolus'] = np.where(result_combined_r['crystalloid_bolus'].notnull(), result_combined_r['crystalloid_bolus'],0)
result_combined_r['colloid_bolus'] = np.where(result_combined_r['colloid_bolus'].notnull(), result_combined_r['colloid_bolus'],0)
result_combined_r['rbc'] = np.where(result_combined_r['rbc'].notnull(), result_combined_r['rbc'],0)
result_combined_r['urine'] = np.where(result_combined_r['urine'].notnull(), result_combined_r['urine'],0)

In [ ]:
result_combined_r = result_combined_r.set_index(['hadm_id','time'])
result_combined_r

sbp        dbp  ...  colloid_bolus_i  urine_i
hadm_id time                                        ...                          
199780  2144-07-05 02:00:00   84.000000  47.000000  ...                0        0
        2144-07-05 03:00:00   90.000000  52.000000  ...                0        0
        2144-07-05 04:00:00   93.000000  39.000000  ...                0        1
        2144-07-05 05:00:00  103.000000  40.000000  ...                0        0
        2144-07-05 06:00:00  106.000000  55.000000  ...                0        0
...                                 ...        ...  ...              ...      ...
100132  2179-03-06 00:00:00   98.000000  55.000000  ...                0        0
        2179-03-06 01:00:00  105.000000  55.500000  ...                0        0
        2179-03-06 02:00:00   88.833333  44.833333  ...                0        0
        2179-03-06 03:00:00   91.000000  44.000000  ...                0        0
        2179-03-06 04:00:00   87.888889  43.222222  ...                0        1

[10200 rows x 14 columns]

In [ ]:
# ## KNN
# from sklearn.impute import KNNImputer

# imputer = KNNImputer(n_neighbors=2)
# after_imputation = imputer.fit_transform(result_combined_r)

# result_combined_KNN = pd.DataFrame(after_imputation)
# result_combined_KNN.columns = result_combined_r.columns
# result_combined_KNN.index = result_combined_r.index
# result_combined_KNN.to_csv("/content/drive/MyDrive/의료인공지능/mimic_test/time_series_non_event_KNN.csv")

In [ ]:
# ## iterative
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# imputer = IterativeImputer()
# after_imputation = imputer.fit_transform(result_combined_r)

# result_combined_iter = pd.DataFrame(after_imputation)
# result_combined_iter.columns = result_combined_r.columns
# result_combined_iter.index = result_combined_r.index
# result_combined_iter.to_csv("/content/drive/MyDrive/의료인공지능/mimic_test/time_series_non_event_iter.csv")

In [ ]:
result_combined_r.to_csv("/content/drive/MyDrive/의료인공지능/mimic_test/time_series_event.csv")

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/의료인공지능/mimic_test/time_series_event.csv')
df2 = pd.read_csv('/content/drive/MyDrive/의료인공지능/mimic_test/time_series_non_event.csv')


result = pd.concat([df1,df2])
result.to_csv("/content/drive/MyDrive/의료인공지능/mimic_test/time_series_total.csv")

In [ ]:
result = result.set_index(['hadm_id','time'])

## Data augmentation

In [ ]:
## KNN
from sklearn.impute import KNNImputer

k=20

imputer = KNNImputer(n_neighbors=k)
after_imputation = imputer.fit_transform(result)

result_combined_KNN = pd.DataFrame(after_imputation)
result_combined_KNN.columns = result.columns
result_combined_KNN.index = result.index
result_combined_KNN.to_csv("/content/drive/MyDrive/의료인공지능/mimic_test/time_series_knn_{x}.csv".format(x=k))

In [ ]:
## iterative
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer()
after_imputation = imputer.fit_transform(result)

result_combined_iter = pd.DataFrame(after_imputation)
result_combined_iter.columns = result.columns
result_combined_iter.index = result.index
result_combined_iter.to_csv("/content/drive/MyDrive/의료인공지능/mimic_test/time_series_iter.csv")

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:638: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
